In [1]:
rm(list=ls())

# 유방암 진단 결과 데이터 셋
### setwd 
setwd("./data/")

In [2]:
### packages
if(!require(class)) install.packages("class"); library(class) # for knn
if(!require(kknn)) install.packages("kknn"); library(kknn) # for wknn
if(!require(caret)) install.packages("caret"); library(caret)
if(!require(e1071)) install.packages("e1071"); library(e1071)
if(!require(dplyr)) install.packages("dplyr"); library(dplyr)

Loading required package: class
Loading required package: kknn
Loading required package: caret
Loading required package: lattice
Loading required package: ggplot2

Attaching package: 'caret'

The following object is masked from 'package:kknn':

    contr.dummy

Loading required package: e1071
Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
### load data
# stringAsFactors = F: read.csv() 함수는 읽어올 때 모든 chr를 factor로 변환하는데
# 이를 자동으로 바꾸지 않고 chr 그대로 읽어오는 것
wdbc <- read.csv('wisc_bc_data.csv', stringsAsFactors = F)
str(wdbc) # 자료구조 확인
dim(wdbc) # 569 x 32
# radius : 반지름 / texture : 텍스처 / perimeter : 둘레 / area : 면적 /
# smmothness : 평활도 compactness : 다짐도 / concavity : 요면 /
# concave points : 요면점 / symmetry : 대칭 / fractal dimension : 프렉탈 차원

'data.frame':	569 obs. of  32 variables:
 $ id               : int  87139402 8910251 905520 868871 9012568 906539 925291 87880 862989 89827 ...
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  1

[1] 569  32

In [4]:
# 불필요한 id 변수 제거
wdbc <- wdbc[-1]
str(wdbc)

'data.frame':	569 obs. of  31 variables:
 $ diagnosis        : chr  "B" "B" "B" "B" ...
 $ radius_mean      : num  12.3 10.6 11 11.3 15.2 ...
 $ texture_mean     : num  12.4 18.9 16.8 13.4 13.2 ...
 $ perimeter_mean   : num  78.8 69.3 70.9 73 97.7 ...
 $ area_mean        : num  464 346 373 385 712 ...
 $ smoothness_mean  : num  0.1028 0.0969 0.1077 0.1164 0.0796 ...
 $ compactness_mean : num  0.0698 0.1147 0.078 0.1136 0.0693 ...
 $ concavity_mean   : num  0.0399 0.0639 0.0305 0.0464 0.0339 ...
 $ points_mean      : num  0.037 0.0264 0.0248 0.048 0.0266 ...
 $ symmetry_mean    : num  0.196 0.192 0.171 0.177 0.172 ...
 $ dimension_mean   : num  0.0595 0.0649 0.0634 0.0607 0.0554 ...
 $ radius_se        : num  0.236 0.451 0.197 0.338 0.178 ...
 $ texture_se       : num  0.666 1.197 1.387 1.343 0.412 ...
 $ perimeter_se     : num  1.67 3.43 1.34 1.85 1.34 ...
 $ area_se          : num  17.4 27.1 13.5 26.3 17.7 ...
 $ smoothness_se    : num  0.00805 0.00747 0.00516 0.01127 0.00501 ...
 $ c

In [5]:
# 종속변수(y) 분포 확인
table(wdbc$diagnosis)
prop.table(table(wdbc$diagnosis))
# B: 양성 / M : 악성
# 악성인 것을 잘 맞춰야한다


  B   M 
357 212 


        B         M 
0.6274165 0.3725835 

In [6]:
# 종속변수(y)인 diagnosis 변수 factor형으로 변환
wdbc$diagnosis <- factor(wdbc$diagnosis, level=c("B","M"))

In [7]:
# set.seed(1) : 난수 고정
set.seed(1)
# createDataPartition
# caret패키지 안에 있는 데이터 파티션, 층화추출
idx <- createDataPartition(y = wdbc$diagnosis, p = 0.7, list =FALSE)
wdbc_train <- wdbc[idx,]
wdbc_test <- wdbc[-idx,]

In [8]:
### 기본 knn (majority voting, 다수결) ###
# 임의의 홀수 k 를 넣어보자
# y를 지우고 넣은것들임, diagnosis 가 y값
# Thus iris %>% head() %>% summary() is equivalent to summary(head(iris))
wdbc_train %>% str
# wdbc_train[-1] diagnosis 빼기
# knn 함수 구조 (학습할 train데이터 독립변수들, preict할 test데이터 독립변수들, 
# 학습할 train 데이터 label, k 개수 지정)
model.knn <- knn(wdbc_train[-1],wdbc_test[-1], wdbc_train$diagnosis, k=5)
confusionMatrix(model.knn, wdbc_test$diagnosis) # Specificity : 0.9048
# 실제로 악성인 것 중에 악성이라고 예측 (Specificity)

'data.frame':	399 obs. of  31 variables:
 $ diagnosis        : Factor w/ 2 levels "B","M": 1 1 1 1 1 1 1 1 1 1 ...
 $ radius_mean      : num  11.3 15.2 11.5 10.5 14.6 ...
 $ texture_mean     : num  13.4 13.2 23.9 19.3 22.7 ...
 $ perimeter_mean   : num  73 97.7 74.5 67.4 96.4 ...
 $ area_mean        : num  385 712 404 336 657 ...
 $ smoothness_mean  : num  0.1164 0.0796 0.0926 0.0999 0.0847 ...
 $ compactness_mean : num  0.1136 0.0693 0.1021 0.0858 0.133 ...
 $ concavity_mean   : num  0.0464 0.0339 0.1112 0.03 0.1029 ...
 $ points_mean      : num  0.048 0.0266 0.0411 0.012 0.0374 ...
 $ symmetry_mean    : num  0.177 0.172 0.139 0.222 0.145 ...
 $ dimension_mean   : num  0.0607 0.0554 0.0657 0.0648 0.0615 ...
 $ radius_se        : num  0.338 0.178 0.239 0.355 0.225 ...
 $ texture_se       : num  1.343 0.412 2.904 1.534 1.108 ...
 $ perimeter_se     : num  1.85 1.34 1.94 2.3 2.22 ...
 $ area_se          : num  26.3 17.7 17 23.1 19.5 ...
 $ smoothness_se    : num  0.01127 0.00501 0.0082 0

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 104   6
         M   3  57
                                          
               Accuracy : 0.9471          
                 95% CI : (0.9019, 0.9755)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.8854          
 Mcnemar's Test P-Value : 0.505           
                                          
            Sensitivity : 0.9720          
            Specificity : 0.9048          
         Pos Pred Value : 0.9455          
         Neg Pred Value : 0.9500          
             Prevalence : 0.6294          
         Detection Rate : 0.6118          
   Detection Prevalence : 0.6471          
      Balanced Accuracy : 0.9384          
                                          
       'Positive' Class : B               
                                          

In [15]:
### weighted knn (가중치) ###
# distance, kernel(가중치)
# 포뮬러, 트레인, 테스트, scale=T는 디폴트 정규화 해주는 것
# scale = T하면 표준화
model.wknn <- kknn(diagnosis ~., wdbc_train, wdbc_test, k=5, scale = F)
## kknn!!
# model.wknn$D : k 인접 5개와의 거리평균?
# model.wknn$fitted.values 예측값

In [16]:
# 속할 확률
model.wknn$prob

B,M
1.0000000,0.0000000
1.0000000,0.0000000
1.0000000,0.0000000
1.0000000,0.0000000
0.0000000,1.0000000
1.0000000,0.0000000
0.0000000,1.0000000
1.0000000,0.0000000
1.0000000,0.0000000
1.0000000,0.0000000


In [17]:
# 최근접 k개와의 거리
model.wknn$D

5.898937,10.68169,14.59813,16.02828,16.34625
16.188467,19.19691,20.64832,21.46105,21.88079
12.644520,16.74894,16.96347,18.44118,20.40876
11.914924,12.60862,14.63993,14.75943,15.62541
89.258070,151.35732,152.47861,163.96004,168.72086
10.761825,14.56224,16.64830,18.47344,20.27516
20.477730,34.29284,52.37920,59.01951,71.51603
14.232023,15.38979,17.64771,17.87372,18.58215
12.629755,15.14727,21.45505,22.57598,22.84454
14.747334,18.30566,20.06200,22.10513,22.68778
83.663987,104.58088,128.00084,131.47425,175.30324


In [18]:
# wknn으로 예측한 값
wknn_pred <- model.wknn$fitted.values
confusionMatrix(wknn_pred, wdbc_test$diagnosis) # Specificity : 0.9524

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 102   3
         M   5  60
                                          
               Accuracy : 0.9529          
                 95% CI : (0.9094, 0.9795)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.8998          
 Mcnemar's Test P-Value : 0.7237          
                                          
            Sensitivity : 0.9533          
            Specificity : 0.9524          
         Pos Pred Value : 0.9714          
         Neg Pred Value : 0.9231          
             Prevalence : 0.6294          
         Detection Rate : 0.6000          
   Detection Prevalence : 0.6176          
      Balanced Accuracy : 0.9528          
                                          
       'Positive' Class : B               
                                          

In [19]:
### Feature scaling ###
summary(wdbc)
# wdbc의 데이터는 데이터의 범주가 제각각
# 변수들의 영향도를 더 정확히 보기 위해서 변수 표준화 혹은 정규화를 한다.

 diagnosis  radius_mean      texture_mean   perimeter_mean     area_mean     
 B:357     Min.   : 6.981   Min.   : 9.71   Min.   : 43.79   Min.   : 143.5  
 M:212     1st Qu.:11.700   1st Qu.:16.17   1st Qu.: 75.17   1st Qu.: 420.3  
           Median :13.370   Median :18.84   Median : 86.24   Median : 551.1  
           Mean   :14.127   Mean   :19.29   Mean   : 91.97   Mean   : 654.9  
           3rd Qu.:15.780   3rd Qu.:21.80   3rd Qu.:104.10   3rd Qu.: 782.7  
           Max.   :28.110   Max.   :39.28   Max.   :188.50   Max.   :2501.0  
 smoothness_mean   compactness_mean  concavity_mean     points_mean     
 Min.   :0.05263   Min.   :0.01938   Min.   :0.00000   Min.   :0.00000  
 1st Qu.:0.08637   1st Qu.:0.06492   1st Qu.:0.02956   1st Qu.:0.02031  
 Median :0.09587   Median :0.09263   Median :0.06154   Median :0.03350  
 Mean   :0.09636   Mean   :0.10434   Mean   :0.08880   Mean   :0.04892  
 3rd Qu.:0.10530   3rd Qu.:0.13040   3rd Qu.:0.13070   3rd Qu.:0.07400  
 Max.   :0.16340

In [20]:
## min-max 스케일링
normalize <- function(x){
  return( (x-min(x))/(max(x)-min(x)) )
}

In [21]:
colnames(wdbc)
wdbc_normal <- as.data.frame(lapply(wdbc[-1], normalize))
summary(wdbc_normal) #  0~1 사이의 값으로 바뀜

[1] "diagnosis"         "radius_mean"       "texture_mean"     
 [4] "perimeter_mean"    "area_mean"         "smoothness_mean"  
 [7] "compactness_mean"  "concavity_mean"    "points_mean"      
[10] "symmetry_mean"     "dimension_mean"    "radius_se"        
[13] "texture_se"        "perimeter_se"      "area_se"          
[16] "smoothness_se"     "compactness_se"    "concavity_se"     
[19] "points_se"         "symmetry_se"       "dimension_se"     
[22] "radius_worst"      "texture_worst"     "perimeter_worst"  
[25] "area_worst"        "smoothness_worst"  "compactness_worst"
[28] "concavity_worst"   "points_worst"      "symmetry_worst"   
[31] "dimension_worst"

  radius_mean      texture_mean    perimeter_mean     area_mean     
 Min.   :0.0000   Min.   :0.0000   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.2233   1st Qu.:0.2185   1st Qu.:0.2168   1st Qu.:0.1174  
 Median :0.3024   Median :0.3088   Median :0.2933   Median :0.1729  
 Mean   :0.3382   Mean   :0.3240   Mean   :0.3329   Mean   :0.2169  
 3rd Qu.:0.4164   3rd Qu.:0.4089   3rd Qu.:0.4168   3rd Qu.:0.2711  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.0000   Max.   :1.0000  
 smoothness_mean  compactness_mean concavity_mean     points_mean    
 Min.   :0.0000   Min.   :0.0000   Min.   :0.00000   Min.   :0.0000  
 1st Qu.:0.3046   1st Qu.:0.1397   1st Qu.:0.06926   1st Qu.:0.1009  
 Median :0.3904   Median :0.2247   Median :0.14419   Median :0.1665  
 Mean   :0.3948   Mean   :0.2606   Mean   :0.20806   Mean   :0.2431  
 3rd Qu.:0.4755   3rd Qu.:0.3405   3rd Qu.:0.30623   3rd Qu.:0.3678  
 Max.   :1.0000   Max.   :1.0000   Max.   :1.00000   Max.   :1.0000  
 symmetry_mean    dimension

In [22]:
wdbc_normal$diagnosis <- wdbc$diagnosis

wdbc_train_n <- wdbc_normal[idx,]
wdbc_test_n <- wdbc_normal[-idx,]
wdbc_train_n[-31]

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
4,0.2034644,0.1244505,0.2018520,0.10235419,0.5756974,0.28900067,0.10859888,0.23836978,0.3590909,0.22662174,...,0.1419424,0.0999467,0.1300862,0.06114825,0.4327412,0.15029446,0.06924121,0.2959107,0.1058545,0.08395645
5,0.3885182,0.1183632,0.3721927,0.24106045,0.2437483,0.15324213,0.07949859,0.13205765,0.3338384,0.11541702,...,0.2942014,0.0988806,0.2693859,0.15579532,0.2735918,0.14204771,0.10878594,0.2810309,0.1817465,0.08277581
7,0.2143499,0.4808928,0.2123557,0.11028632,0.3609280,0.25372677,0.26054358,0.20402584,0.1656566,0.33150800,...,0.1618641,0.6700426,0.1587230,0.07102831,0.3871756,0.21772371,0.28993610,0.3317182,0.1078257,0.21172767
9,0.1660751,0.3239770,0.1632230,0.08169671,0.4266498,0.20366849,0.07017338,0.05969185,0.5843434,0.31276327,...,0.1284240,0.3009062,0.1185816,0.05348014,0.3350063,0.11769557,0.06379393,0.1100687,0.2485709,0.13433032
16,0.3601212,0.4386202,0.3634856,0.21785790,0.2897897,0.34850623,0.24109653,0.18568588,0.1989899,0.24241786,...,0.2685877,0.4064499,0.2763584,0.13475718,0.2075546,0.28117511,0.29249201,0.3797251,0.1366056,0.16397744
18,0.2692508,0.2566791,0.2588626,0.14693531,0.4529205,0.17563340,0.07481256,0.10472167,0.3267677,0.20808762,...,0.2056208,0.2420043,0.1906967,0.09565474,0.4439675,0.09111195,0.08146965,0.1925086,0.2213680,0.09077791
20,0.2465332,0.1210687,0.2438670,0.13247084,0.4872258,0.23222502,0.06689316,0.14324056,0.4141414,0.31065712,...,0.1924582,0.1543177,0.2040440,0.08847818,0.4723635,0.22432110,0.07919329,0.2813402,0.3753203,0.24511347
21,0.2238156,0.2529591,0.2134614,0.11741251,0.4072402,0.12891847,0.08924555,0.16098410,0.2303030,0.23146588,...,0.1807186,0.2497335,0.1693809,0.08265336,0.4036849,0.07442443,0.12148562,0.3776632,0.1985019,0.10448642
22,0.2228690,0.4981400,0.2251399,0.11151644,0.6406969,0.41660021,0.10578725,0.22519881,0.5409091,0.50737152,...,0.1796514,0.5375800,0.1778475,0.07434624,0.6975500,0.28893675,0.11142173,0.4494845,0.2440371,0.29292929
24,0.3842586,0.2259046,0.3855297,0.22523860,0.5630586,0.49481627,0.26663543,0.42415507,0.4752525,0.30981466,...,0.2909996,0.1681770,0.2763584,0.14190916,0.4452883,0.25323321,0.15654952,0.4890034,0.2020501,0.14921947


In [23]:
### 기본 knn (majority voting) ###
# 임의의 홀수 k 를 넣어보자
wdbc_pred_n <- knn(wdbc_train_n[-31],wdbc_test_n[-31], wdbc_train_n$diagnosis, k=5)
confusionMatrix(wdbc_pred_n, wdbc_test$diagnosis) # Specificity : 0.9524

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 105   3
         M   2  60
                                          
               Accuracy : 0.9706          
                 95% CI : (0.9327, 0.9904)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9367          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9813          
            Specificity : 0.9524          
         Pos Pred Value : 0.9722          
         Neg Pred Value : 0.9677          
             Prevalence : 0.6294          
         Detection Rate : 0.6176          
   Detection Prevalence : 0.6353          
      Balanced Accuracy : 0.9668          
                                          
       'Positive' Class : B               
                                          

In [24]:
### weighted knn (가중치) ###
# distance, kernel(가중치)
wknn.model <- kknn(diagnosis ~., wdbc_train_n, wdbc_test_n, k=5, scale = F)
wdbc_pred_n2 <- wknn.model$fitted.values
confusionMatrix(wdbc_pred_n2, wdbc_test$diagnosis) # Specificity : 0.9524

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 104   3
         M   3  60
                                          
               Accuracy : 0.9647          
                 95% CI : (0.9248, 0.9869)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9243          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9720          
            Specificity : 0.9524          
         Pos Pred Value : 0.9720          
         Neg Pred Value : 0.9524          
             Prevalence : 0.6294          
         Detection Rate : 0.6118          
   Detection Prevalence : 0.6294          
      Balanced Accuracy : 0.9622          
                                          
       'Positive' Class : B               
                                          

In [24]:
### 최적의 K를 구하는 방법 ###
# Cross-Validation
# ks = 실험할 k, 일부러 홀수로 지정하였다.
wdbc.cv <- train.kknn(diagnosis ~., wdbc_train_n, 
                      ks = seq(1, 50, by=2), scale = T);wdbc.cv
best_k <- wdbc.cv$best.parameters$k;best_k

wdbc_pred_cv <- kknn(diagnosis ~., train = wdbc_train_n, test = wdbc_test_n, k = best_k, scale = F)
wdbc_pred_cv <- wdbc_pred_cv$fitted.values

confusionMatrix(wdbc_pred_cv, wdbc_test$diagnosis) # Specificity : 0.9524


Call:
train.kknn(formula = diagnosis ~ ., data = wdbc_train_n, ks = seq(1,     50, by = 2), scale = T)

Type of response variable: nominal
Minimal misclassification: 0.03258145
Best kernel: optimal
Best k: 27

[1] 27

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 106   3
         M   1  60
                                          
               Accuracy : 0.9765          
                 95% CI : (0.9409, 0.9936)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9492          
 Mcnemar's Test P-Value : 0.6171          
                                          
            Sensitivity : 0.9907          
            Specificity : 0.9524          
         Pos Pred Value : 0.9725          
         Neg Pred Value : 0.9836          
             Prevalence : 0.6294          
         Detection Rate : 0.6235          
   Detection Prevalence : 0.6412          
      Balanced Accuracy : 0.9715          
                                          
       'Positive' Class : B               
                                          

In [25]:
##########################
#### caret 패키지로 돌려보기!
### 사용 가능한 모델
## https://topepo.github.io/caret/available-models.html

######### cross validation
cv <- trainControl(method = "cv", number = 5, verbose = T)
#변수의 계수를 구하는건가? 5번의 샘플링을 하겠다. 
#verbose는 caret으로 train 하는 경우 적용하면됨
repCv <- trainControl(method = "repeatedcv", number = 5,repeats = 3, verbose = T)
# 5번 리샘플링하는 과정을 3번 적용하겟다. 5-fold 3 repeat

In [26]:
######### knn
#### tune parameter
knn.grid = expand.grid(
  .k = c(1,3,5,7,9)
)
# k개수 지정해주기

In [31]:
wdbc_train_n

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst,diagnosis
4,0.2034644,0.1244505,0.2018520,0.10235419,0.5756974,0.28900067,0.10859888,0.23836978,0.3590909,0.22662174,...,0.0999467,0.1300862,0.06114825,0.4327412,0.15029446,0.06924121,0.2959107,0.1058545,0.08395645,B
5,0.3885182,0.1183632,0.3721927,0.24106045,0.2437483,0.15324213,0.07949859,0.13205765,0.3338384,0.11541702,...,0.0988806,0.2693859,0.15579532,0.2735918,0.14204771,0.10878594,0.2810309,0.1817465,0.08277581,B
7,0.2143499,0.4808928,0.2123557,0.11028632,0.3609280,0.25372677,0.26054358,0.20402584,0.1656566,0.33150800,...,0.6700426,0.1587230,0.07102831,0.3871756,0.21772371,0.28993610,0.3317182,0.1078257,0.21172767,B
9,0.1660751,0.3239770,0.1632230,0.08169671,0.4266498,0.20366849,0.07017338,0.05969185,0.5843434,0.31276327,...,0.3009062,0.1185816,0.05348014,0.3350063,0.11769557,0.06379393,0.1100687,0.2485709,0.13433032,B
16,0.3601212,0.4386202,0.3634856,0.21785790,0.2897897,0.34850623,0.24109653,0.18568588,0.1989899,0.24241786,...,0.4064499,0.2763584,0.13475718,0.2075546,0.28117511,0.29249201,0.3797251,0.1366056,0.16397744,B
18,0.2692508,0.2566791,0.2588626,0.14693531,0.4529205,0.17563340,0.07481256,0.10472167,0.3267677,0.20808762,...,0.2420043,0.1906967,0.09565474,0.4439675,0.09111195,0.08146965,0.1925086,0.2213680,0.09077791,B
20,0.2465332,0.1210687,0.2438670,0.13247084,0.4872258,0.23222502,0.06689316,0.14324056,0.4141414,0.31065712,...,0.1543177,0.2040440,0.08847818,0.4723635,0.22432110,0.07919329,0.2813402,0.3753203,0.24511347,B
21,0.2238156,0.2529591,0.2134614,0.11741251,0.4072402,0.12891847,0.08924555,0.16098410,0.2303030,0.23146588,...,0.2497335,0.1693809,0.08265336,0.4036849,0.07442443,0.12148562,0.3776632,0.1985019,0.10448642,B
22,0.2228690,0.4981400,0.2251399,0.11151644,0.6406969,0.41660021,0.10578725,0.22519881,0.5409091,0.50737152,...,0.5375800,0.1778475,0.07434624,0.6975500,0.28893675,0.11142173,0.4494845,0.2440371,0.29292929,B
24,0.3842586,0.2259046,0.3855297,0.22523860,0.5630586,0.49481627,0.26663543,0.42415507,0.4752525,0.30981466,...,0.1681770,0.2763584,0.14190916,0.4452883,0.25323321,0.15654952,0.4890034,0.2020501,0.14921947,B


In [28]:
train.knn <- train(diagnosis~.,wdbc_train_n, method = "knn",trControl = cv,
                   tuneGrid = knn.grid)
train.knn$results
train.knn$bestTune
# 학습하고 결과보기

+ Fold1: k=1 
- Fold1: k=1 
+ Fold1: k=3 
- Fold1: k=3 
+ Fold1: k=5 
- Fold1: k=5 
+ Fold1: k=7 
- Fold1: k=7 
+ Fold1: k=9 
- Fold1: k=9 
+ Fold2: k=1 
- Fold2: k=1 
+ Fold2: k=3 
- Fold2: k=3 
+ Fold2: k=5 
- Fold2: k=5 
+ Fold2: k=7 
- Fold2: k=7 
+ Fold2: k=9 
- Fold2: k=9 
+ Fold3: k=1 
- Fold3: k=1 
+ Fold3: k=3 
- Fold3: k=3 
+ Fold3: k=5 
- Fold3: k=5 
+ Fold3: k=7 
- Fold3: k=7 
+ Fold3: k=9 
- Fold3: k=9 
+ Fold4: k=1 
- Fold4: k=1 
+ Fold4: k=3 
- Fold4: k=3 
+ Fold4: k=5 
- Fold4: k=5 
+ Fold4: k=7 
- Fold4: k=7 
+ Fold4: k=9 
- Fold4: k=9 
+ Fold5: k=1 
- Fold5: k=1 
+ Fold5: k=3 
- Fold5: k=3 
+ Fold5: k=5 
- Fold5: k=5 
+ Fold5: k=7 
- Fold5: k=7 
+ Fold5: k=9 
- Fold5: k=9 
Aggregating results
Selecting tuning parameters
Fitting k = 3 on full training set


k,Accuracy,Kappa,AccuracySD,KappaSD
1,0.9499367,0.8928165,0.015180228,0.03327482
3,0.9674684,0.9300767,0.018915542,0.04092240
5,0.9624367,0.9189347,0.008727381,0.01898113
7,0.9624367,0.9188670,0.012421440,0.02710452
9,0.9624367,0.9188670,0.012421440,0.02710452


,k
2,3


In [29]:
predict.knn <- predict(train.knn,wdbc_test_n)
confusionMatrix(predict.knn, wdbc_test$diagnosis)

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 104   3
         M   3  60
                                          
               Accuracy : 0.9647          
                 95% CI : (0.9248, 0.9869)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9243          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.9720          
            Specificity : 0.9524          
         Pos Pred Value : 0.9720          
         Neg Pred Value : 0.9524          
             Prevalence : 0.6294          
         Detection Rate : 0.6118          
   Detection Prevalence : 0.6294          
      Balanced Accuracy : 0.9622          
                                          
       'Positive' Class : B               
                                          

In [37]:
######### wknn
#### tune parameter
wknn.grid = expand.grid(
  .kmax = c(1,3,5),
  .distance = c(1,2),
  .kernel = "optimal"
)
# k개 1,3,5 개 / 유클리드 맨하튼 다해봐라?
train.wknn <- train(diagnosis~.,wdbc_train_n, method = "kknn", trControl = cv,
                    tuneGrid = wknn.grid)
predict.wknn <- predict(train.wknn,wdbc_test_n)
confusionMatrix(predict.wknn, wdbc_test$diagnosis)

+ Fold1: kmax=1, distance=1, kernel=optimal 
- Fold1: kmax=1, distance=1, kernel=optimal 
+ Fold1: kmax=3, distance=1, kernel=optimal 
- Fold1: kmax=3, distance=1, kernel=optimal 
+ Fold1: kmax=5, distance=1, kernel=optimal 
- Fold1: kmax=5, distance=1, kernel=optimal 
+ Fold1: kmax=1, distance=2, kernel=optimal 
- Fold1: kmax=1, distance=2, kernel=optimal 
+ Fold1: kmax=3, distance=2, kernel=optimal 
- Fold1: kmax=3, distance=2, kernel=optimal 
+ Fold1: kmax=5, distance=2, kernel=optimal 
- Fold1: kmax=5, distance=2, kernel=optimal 
+ Fold2: kmax=1, distance=1, kernel=optimal 
- Fold2: kmax=1, distance=1, kernel=optimal 
+ Fold2: kmax=3, distance=1, kernel=optimal 
- Fold2: kmax=3, distance=1, kernel=optimal 
+ Fold2: kmax=5, distance=1, kernel=optimal 
- Fold2: kmax=5, distance=1, kernel=optimal 
+ Fold2: kmax=1, distance=2, kernel=optimal 
- Fold2: kmax=1, distance=2, kernel=optimal 
+ Fold2: kmax=3, distance=2, kernel=optimal 
- Fold2: kmax=3, distance=2, kernel=optimal 
+ Fold2: k

Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 105   4
         M   2  59
                                          
               Accuracy : 0.9647          
                 95% CI : (0.9248, 0.9869)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9238          
 Mcnemar's Test P-Value : 0.6831          
                                          
            Sensitivity : 0.9813          
            Specificity : 0.9365          
         Pos Pred Value : 0.9633          
         Neg Pred Value : 0.9672          
             Prevalence : 0.6294          
         Detection Rate : 0.6176          
   Detection Prevalence : 0.6412          
      Balanced Accuracy : 0.9589          
                                          
       'Positive' Class : B               
                                          

In [33]:
######### lda
train.lda <- train(diagnosis~.,wdbc_train_n, method = "lda", trControl = repCv)
predict.lda <- predict(train.lda,wdbc_test_n)
confusionMatrix(predict.lda, wdbc_test$diagnosis)

+ Fold1.Rep1: parameter=none 
- Fold1.Rep1: parameter=none 
+ Fold2.Rep1: parameter=none 
- Fold2.Rep1: parameter=none 
+ Fold3.Rep1: parameter=none 
- Fold3.Rep1: parameter=none 
+ Fold4.Rep1: parameter=none 
- Fold4.Rep1: parameter=none 
+ Fold5.Rep1: parameter=none 
- Fold5.Rep1: parameter=none 
+ Fold1.Rep2: parameter=none 
- Fold1.Rep2: parameter=none 
+ Fold2.Rep2: parameter=none 
- Fold2.Rep2: parameter=none 
+ Fold3.Rep2: parameter=none 
- Fold3.Rep2: parameter=none 
+ Fold4.Rep2: parameter=none 
- Fold4.Rep2: parameter=none 
+ Fold5.Rep2: parameter=none 
- Fold5.Rep2: parameter=none 
+ Fold1.Rep3: parameter=none 
- Fold1.Rep3: parameter=none 
+ Fold2.Rep3: parameter=none 
- Fold2.Rep3: parameter=none 
+ Fold3.Rep3: parameter=none 
- Fold3.Rep3: parameter=none 
+ Fold4.Rep3: parameter=none 
- Fold4.Rep3: parameter=none 
+ Fold5.Rep3: parameter=none 
- Fold5.Rep3: parameter=none 
Aggregating results
Fitting final model on full training set


Confusion Matrix and Statistics

          Reference
Prediction   B   M
         B 106   4
         M   1  59
                                          
               Accuracy : 0.9706          
                 95% CI : (0.9327, 0.9904)
    No Information Rate : 0.6294          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9363          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.9907          
            Specificity : 0.9365          
         Pos Pred Value : 0.9636          
         Neg Pred Value : 0.9833          
             Prevalence : 0.6294          
         Detection Rate : 0.6235          
   Detection Prevalence : 0.6471          
      Balanced Accuracy : 0.9636          
                                          
       'Positive' Class : B               
                                          

In [35]:
wdbc_train_n
str(wdbc_train_n)

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst,diagnosis
4,0.2034644,0.1244505,0.2018520,0.10235419,0.5756974,0.28900067,0.10859888,0.23836978,0.3590909,0.22662174,...,0.0999467,0.1300862,0.06114825,0.4327412,0.15029446,0.06924121,0.2959107,0.1058545,0.08395645,B
5,0.3885182,0.1183632,0.3721927,0.24106045,0.2437483,0.15324213,0.07949859,0.13205765,0.3338384,0.11541702,...,0.0988806,0.2693859,0.15579532,0.2735918,0.14204771,0.10878594,0.2810309,0.1817465,0.08277581,B
7,0.2143499,0.4808928,0.2123557,0.11028632,0.3609280,0.25372677,0.26054358,0.20402584,0.1656566,0.33150800,...,0.6700426,0.1587230,0.07102831,0.3871756,0.21772371,0.28993610,0.3317182,0.1078257,0.21172767,B
9,0.1660751,0.3239770,0.1632230,0.08169671,0.4266498,0.20366849,0.07017338,0.05969185,0.5843434,0.31276327,...,0.3009062,0.1185816,0.05348014,0.3350063,0.11769557,0.06379393,0.1100687,0.2485709,0.13433032,B
16,0.3601212,0.4386202,0.3634856,0.21785790,0.2897897,0.34850623,0.24109653,0.18568588,0.1989899,0.24241786,...,0.4064499,0.2763584,0.13475718,0.2075546,0.28117511,0.29249201,0.3797251,0.1366056,0.16397744,B
18,0.2692508,0.2566791,0.2588626,0.14693531,0.4529205,0.17563340,0.07481256,0.10472167,0.3267677,0.20808762,...,0.2420043,0.1906967,0.09565474,0.4439675,0.09111195,0.08146965,0.1925086,0.2213680,0.09077791,B
20,0.2465332,0.1210687,0.2438670,0.13247084,0.4872258,0.23222502,0.06689316,0.14324056,0.4141414,0.31065712,...,0.1543177,0.2040440,0.08847818,0.4723635,0.22432110,0.07919329,0.2813402,0.3753203,0.24511347,B
21,0.2238156,0.2529591,0.2134614,0.11741251,0.4072402,0.12891847,0.08924555,0.16098410,0.2303030,0.23146588,...,0.2497335,0.1693809,0.08265336,0.4036849,0.07442443,0.12148562,0.3776632,0.1985019,0.10448642,B
22,0.2228690,0.4981400,0.2251399,0.11151644,0.6406969,0.41660021,0.10578725,0.22519881,0.5409091,0.50737152,...,0.5375800,0.1778475,0.07434624,0.6975500,0.28893675,0.11142173,0.4494845,0.2440371,0.29292929,B
24,0.3842586,0.2259046,0.3855297,0.22523860,0.5630586,0.49481627,0.26663543,0.42415507,0.4752525,0.30981466,...,0.1681770,0.2763584,0.14190916,0.4452883,0.25323321,0.15654952,0.4890034,0.2020501,0.14921947,B


'data.frame':	399 obs. of  31 variables:
 $ radius_mean      : num  0.203 0.389 0.214 0.166 0.36 ...
 $ texture_mean     : num  0.124 0.118 0.481 0.324 0.439 ...
 $ perimeter_mean   : num  0.202 0.372 0.212 0.163 0.363 ...
 $ area_mean        : num  0.1024 0.2411 0.1103 0.0817 0.2179 ...
 $ smoothness_mean  : num  0.576 0.244 0.361 0.427 0.29 ...
 $ compactness_mean : num  0.289 0.153 0.254 0.204 0.349 ...
 $ concavity_mean   : num  0.1086 0.0795 0.2605 0.0702 0.2411 ...
 $ points_mean      : num  0.2384 0.1321 0.204 0.0597 0.1857 ...
 $ symmetry_mean    : num  0.359 0.334 0.166 0.584 0.199 ...
 $ dimension_mean   : num  0.227 0.115 0.332 0.313 0.242 ...
 $ radius_se        : num  0.0822 0.0242 0.0461 0.0882 0.0412 ...
 $ texture_se       : num  0.2172 0.0116 0.5622 0.2594 0.1653 ...
 $ perimeter_se     : num  0.0515 0.0274 0.0556 0.0728 0.0691 ...
 $ area_se          : num  0.0365 0.0204 0.019 0.0305 0.0238 ...
 $ smoothness_se    : num  0.325 0.112 0.221 0.2 0.086 ...
 $ compactness_

In [ ]:
######### logistic
train.glm <- train(diagnosis~.,wdbc_train_n, method = "glm", trControl =cv)
predict.glm <- predict(train.glm,wdbc_test_n)
confusionMatrix(predict.glm, wdbc_test$diagnosis)